In [ ]:
%pip install python-dotenv
%pip install langchain_openai

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [4]:

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-2b0d2b06-f

In [ ]:
! pip install langchain_community

In [14]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)


## Conversation Summary
Now let's take a look at using a slightly more complex type of memory - ConversationSummaryMemory. This type of memory creates a summary of the conversation over time. This can be useful for condensing information from the conversation over time. Conversation summary memory summarizes the conversation as it happens and stores the current summary in memory. This memory can then be used to inject the summary of the conversation so far into a prompt/chain. This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [5]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

In [6]:
memory = ConversationSummaryMemory(llm= model)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [7]:
memory.load_memory_variables({})

{'history': 'The human says hello, and the AI responds with "what\'s up."'}

In [8]:
memory = ConversationSummaryMemory(llm= model, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [9]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human greets the AI with "hi," and the AI responds with "what\'s up."')]}

In [10]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'The human greets the AI, and the AI responds with a casual greeting.'

## Initializing with messages/existing summary

In [11]:
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

In [12]:
memory = ConversationSummaryMemory.from_messages(
    llm= model,
    chat_memory=history,
    return_messages=True
)

In [13]:
memory.buffer


'The human greets the AI, and the AI responds with a greeting.'

Optionally you can speed up initialization using a previously generated summary, and avoid regenerating the summary by just initializing directly.

In [33]:
existing_summary="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential."

memory = ConversationSummaryMemory(
    llm= model,
)

In [34]:
memory.buffer

''

In [35]:

new_summary = memory.predict_new_summary(messages, existing_summary= existing_summary)

new_summary

'The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human greets the AI, and the AI responds casually.'

In [36]:
len(new_summary)

230

In [43]:

from langchain.agents import Agent, ConversationalAgent
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory, ConversationBufferMemory

In [40]:

memory = ConversationSummaryMemory(
    llm= model,
)

In [46]:
agent = ConversationalAgent(
    llm_chain= model,
    memory= memory
)

ValidationError: 1 validation error for ConversationalAgent
llm_chain
  field required (type=value_error.missing)